In [16]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token:str, **kwargs) -> None:
        print(f"Received streaming token {token}, kwargs:{kwargs}")
        
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Received non-streaming response: {response}")

In [17]:
from typing import List
from langchain_core.language_models.chat_models import BaseChatModel
from langchain.schema.messages import BaseMessage, AIMessage, HumanMessage
from langchain.schema.output import ChatResult, ChatGeneration
from langchain_core.callbacks import Callbacks
from llmstudio.llm import LLM
from typing import Any, Optional
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field

class ChatLLMstudio(ChatOpenAI):
    model_id: str
    llm: LLM = Field(default=None)

    def __init__(self, model_id: str, **kwargs):
        super().__init__(model_id=model_id, **kwargs)
        self.llm = LLM(model_id, **kwargs)
    
    @property
    def _llm_type(self):
        return "LLMstudio"

    def _generate(self, messages: List[BaseMessage], **kwargs) -> ChatResult:
        messages_dicts, _ = self._create_message_dicts(messages, [])
        response = self.llm.chat(messages_dicts, **kwargs)
        return self._create_chat_result(response)

In [18]:
gpt3 = ChatLLMstudio(model_id='openai/gpt-3.5-turbo', callbacks=[MyCustomHandler()])

In [19]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

@tool
def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."


def assistant(question: str)->str:
    """YOUR CODE HERE"""
    tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, gpt3, agent=AgentType.OPENAI_MULTI_FUNCTIONS, handle_parsing_errors="Check your output and make sure it conforms!", verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [20]:
assistant('When does my train depart? My ticket number is 1234')




> Entering new AgentExecutor chain...
Received non-streaming response: generations=[[ChatGeneration(generation_info={'finish_reason': 'function_call', 'logprobs': None}, message=AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"actions":[{"action_name":"get_departure","action":{"ticket_number":"1234"}}]}', 'name': 'tool_selection'}}))]] llm_output={'token_usage': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None} run=None

Invoking: `get_departure` with `{'ticket_number': '1234'}`


12:00 AMReceived non-streaming response: generations=[[ChatGeneration(text='Your train departs at 12:00 AM. If you need any more assistance, feel free to ask!', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Your train departs at 12:00 AM. If you need any more assistance, feel free to ask!'))]] llm_output={'token_usage': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None} run=None
Your train departs at 12:00 AM. I

{'input': 'When does my train depart? My ticket number is 1234',
 'output': 'Your train departs at 12:00 AM. If you need any more assistance, feel free to ask!'}